In [34]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [35]:
file_path = "./jsonl/detailed/batch_Ex6AXhII2FSLXy3feLxn6BXz_output.jsonl"
file_path = "./jsonl/detailed/batch_SHll77JlKuqCvEl1AjhX8dmY_output.jsonl"
gt_data = pd.read_json(f"./images_detailed//configurations.json")

In [36]:
# Dictionary to map custom_id to message content
custom_id_to_content = {}

# Open the file and read line by line
with open(file_path, "r") as file:
    for line in file:
        json_obj = json.loads(line)
        # Extract custom_id and message content
        custom_id = json_obj.get("custom_id")
        message_content = json_obj["response"]["body"]["choices"][0]["message"][
            "content"
        ]
        # Map custom_id to message content
        if custom_id:
            custom_id_to_content[custom_id.replace("uid__", "")] = message_content

# Print the dictionary to verify
print(len(custom_id_to_content))

150


In [37]:
gt_data = (
    gt_data.T
)  # use indexx as a new column in the dataframe and called it image_path
gt_data = gt_data.reset_index()
gt_data = gt_data.rename(columns={"index": "image_path"})

# ad a new column to gt_data with content of  custom_id_to_content (match key with image_path)
gt_data["message"] = gt_data["image_path"].map(custom_id_to_content)

In [38]:
gt_data

,image_path,num_points,num_intersections,line_colors,line_thickness,coordinates,message
0,image_0_thickness_2.png,3,0,"[red, blue]",2,"{'line1': [[0.0, 7.596981558052801], [5.0, 3.7...",The blue and red line plots cross each other t...
1,image_0_thickness_3.png,3,0,"[red, blue]",3,"{'line1': [[0.0, 7.596981558052801], [5.0, 3.7...",The blue and red line plots cross each other o...
2,image_0_thickness_4.png,3,0,"[red, blue]",4,"{'line1': [[0.0, 7.596981558052801], [5.0, 3.7...",The blue and red line plots cross each other t...
3,image_1_thickness_2.png,3,1,"[red, blue]",2,"{'line1': [[0.0, 6.015752719221071], [5.0, 7.9...",The blue and red line plots cross each other t...
4,image_1_thickness_3.png,3,1,"[red, blue]",3,"{'line1': [[0.0, 6.015752719221071], [5.0, 7.9...",The blue and red line plots cross each other t...
...,...,...,...,...,...,...,...
145,image_48_thickness_3.png,3,2,"[red, blue]",3,"{'line1': [[0.0, 9.69080313319073], [5.0, 2.36...",The blue and red line plots cross each other t...
146,image_48_thickness_4.png,3,2,"[red, blue]",4,"{'line1': [[0.0, 9.69080313319073], [5.0, 2.36...",The blue and red line plots cross each other t...
147,image_49_thickness_2.png,3,2,"[red, blue]",2,"{'line1': [[0.0, 4.201544340964238], [5.0, 9.7...",The blue and red line plots cross each other t...
148,image_49_thickness_3.png,3,2,"[red, blue]",3,"{'line1': [[0.0, 4.201544340964238], [5.0, 9.7...",The blue and red line plots cross each other t...


In [39]:
# remove nan
gt_data = gt_data.dropna()

In [40]:
import re


def extract_marked_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Regular expression to find numbers (digits or words)
    number_words = "zero|one|two|three|four|five|six|seven|eight|nine|ten"
    pattern = rf"\b({number_words}|\d+)\b"

    # Mapping from words to digits
    number_map = {
        "zero": "0",
        "one": "1",
        "two": "2",
        "three": "3",
        "four": "4",
        "five": "5",
        "six": "6",
        "seven": "7",
        "eight": "8",
        "nine": "9",
        "ten": "10",
    }

    # Search for the first occurrence of a number
    match = re.search(pattern, text)
    if match:
        # Map the word to its corresponding digit if it's a word
        return number_map.get(match.group(0), match.group(0))
    else:
        # Check for implied single intersection
        if "intersect once" in text or "intersect only once" in text:
            return "1"
        # Check for no intersections
        elif (
            "do not intersect" in text
            or "not intersect" in text
            or "no intersection" in text
        ):
            return "0"
        # Check for multiple intersections
        elif "intersect twice" in text:
            return "2"
        elif "intersect three times" in text:
            return "3"
        # Default case if no clear intersection information is found
        elif "once" in text:
            return "1"

    if "do not cross" in text:
        return "0"
    elif "twice" in text:
        return "2"
    elif "three times" in text:
        return "3"
    elif "single crossing" in text:
        return "1"
    elif "never cross" in text:
        return "0"
    else:
        return "number_not_found" + text


gt_data["extracted_number"] = gt_data["message"].apply(extract_marked_text)

In [41]:
gt_data["extracted_number"].value_counts()

extracted_number
2    81
1    59
3     8
0     2
Name: count, dtype: int64

In [42]:
# count none or na in gt_data extracted_number
gt_data["extracted_number"].isna().sum()
# show their value in the message (same row)
gt_data[gt_data["extracted_number"].isna()]["message"]

Series([], Name: message, dtype: object)

In [43]:
# Cast num_intersections to integer
gt_data["num_intersections"] = gt_data["num_intersections"].astype(int)

# Cast extracted_number to integer (it should already be integer from the extraction function, but just to ensure consistency)
gt_data["extracted_number"] = gt_data["extracted_number"].astype(int)

# Compare num_intersections and extracted_number to check if they are the same
gt_data["is_correct"] = gt_data["num_intersections"] == gt_data["extracted_number"]

# Display the DataFrame with the new column
gt_data[["image_path", "num_intersections", "extracted_number", "is_correct"]]

,image_path,num_intersections,extracted_number,is_correct
0,image_0_thickness_2.png,0,2,False
1,image_0_thickness_3.png,0,1,False
2,image_0_thickness_4.png,0,2,False
3,image_1_thickness_2.png,1,2,False
4,image_1_thickness_3.png,1,2,False
...,...,...,...,...
145,image_48_thickness_3.png,2,2,True
146,image_48_thickness_4.png,2,2,True
147,image_49_thickness_2.png,2,2,True
148,image_49_thickness_3.png,2,2,True


In [44]:
gt_data["is_correct"].value_counts()

is_correct
False    75
True     75
Name: count, dtype: int64

In [45]:
100 * np.mean(gt_data["is_correct"])

50.0

In [46]:
gt_data

,image_path,num_points,num_intersections,line_colors,line_thickness,coordinates,message,extracted_number,is_correct
0,image_0_thickness_2.png,3,0,"[red, blue]",2,"{'line1': [[0.0, 7.596981558052801], [5.0, 3.7...",The blue and red line plots cross each other t...,2,False
1,image_0_thickness_3.png,3,0,"[red, blue]",3,"{'line1': [[0.0, 7.596981558052801], [5.0, 3.7...",The blue and red line plots cross each other o...,1,False
2,image_0_thickness_4.png,3,0,"[red, blue]",4,"{'line1': [[0.0, 7.596981558052801], [5.0, 3.7...",The blue and red line plots cross each other t...,2,False
3,image_1_thickness_2.png,3,1,"[red, blue]",2,"{'line1': [[0.0, 6.015752719221071], [5.0, 7.9...",The blue and red line plots cross each other t...,2,False
4,image_1_thickness_3.png,3,1,"[red, blue]",3,"{'line1': [[0.0, 6.015752719221071], [5.0, 7.9...",The blue and red line plots cross each other t...,2,False
...,...,...,...,...,...,...,...,...,...
145,image_48_thickness_3.png,3,2,"[red, blue]",3,"{'line1': [[0.0, 9.69080313319073], [5.0, 2.36...",The blue and red line plots cross each other t...,2,True
146,image_48_thickness_4.png,3,2,"[red, blue]",4,"{'line1': [[0.0, 9.69080313319073], [5.0, 2.36...",The blue and red line plots cross each other t...,2,True
147,image_49_thickness_2.png,3,2,"[red, blue]",2,"{'line1': [[0.0, 4.201544340964238], [5.0, 9.7...",The blue and red line plots cross each other t...,2,True
148,image_49_thickness_3.png,3,2,"[red, blue]",3,"{'line1': [[0.0, 4.201544340964238], [5.0, 9.7...",The blue and red line plots cross each other t...,2,True


In [47]:
# Assuming 'gt_data' DataFrame has columns 'line_thickness', 'num_intersections', and 'extracted_number'

# Convert line_thickness to an appropriate numeric type if necessary
gt_data["line_thickness"] = gt_data["line_thickness"].astype(int)

# Calculate accuracy for each thickness
accuracy_by_thickness = gt_data.groupby("line_thickness").apply(
    lambda df: (df["num_intersections"] == df["extracted_number"]).mean()
)

# Print the accuracy by line thickness
# print(accuracy_by_thickness
#
# #)
# multiply by 100 and round 2 decimal
accuracy_by_thickness = 100 * accuracy_by_thickness.round(2)

In [48]:
print(accuracy_by_thickness.to_latex())

\begin{tabular}{lr}
\toprule
 & 0 \\
line_thickness &  \\
\midrule
2 & 52.000000 \\
3 & 46.000000 \\
4 & 52.000000 \\
\bottomrule
\end{tabular}



# Export

In [49]:
gt_data["Model"] = ["gpt-4o"] * len(gt_data)
# rename message to model_output_raw
gt_data.rename(columns={"message": "model_output_raw"}, inplace=True)
gt_data.to_pickle("./data/gpt-4o-detailed_1.pkl")